This Jupyter file reads our analysis files and for each repository appends the number of contributors in a repository to each entry in the analysis file

In [ ]:
import json
json_file = open('analysis_java_worst.json')
json_str = json_file.read()
json_data = json.loads(json_str)

In [ ]:
from github import Github, RateLimitExceededException, UnknownObjectException

# Read Token
access_token = open('GITHUB_TOKEN.txt', 'r').read()
user = Github(access_token)
user.per_page = 100

In [ ]:
def check_rate_limit():
    rate_limit = user.get_rate_limit()
    remaining = rate_limit.search.remaining
    
    if remaining > 0:
        return
    
    reset_time = rate_limit.search.reset
    now = datetime.utcnow()

    if now < reset_time:
        sleep_time = (reset_time - now).total_seconds()
        print('=> Waiting {} seconds for rate limit reset.'.format(sleep_time))
        time.sleep(sleep_time)

In [ ]:
for i in range (0, len(json_data)):
    print("================ {} ================".format(i))
    try:
        repo = user.get_repo(json_data[i]['owner']+'/'+json_data[i]['name'])
        contributor_count = repo.get_contributors().totalCount
        
        json_data[i]['contributor_count'] = contributor_count
        
        with open("./contr_java_worst.json", 'a+') as f:
            f.write(json.dumps(json_data[i], indent=4))
            f.write(',')  
        
    except RateLimitExceededException:
        check_rate_limit()
        continue
    except UnknownObjectException:
        
        json_data[i]['contributor_count'] = -1
        
        with open("./contr_java_worst.json", 'a+') as f:
            f.write(json.dumps(json_data[i], indent=4))
            f.write(',')
        